In [39]:
import warnings
warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
import sklearn
import dtale
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier,AdaBoostClassifier
from sklearn.model_selection import GridSearchCV 


In [2]:
geom_varbs=[
'Street type'
,'Identifying feature type'
,'School zone location'
,'School zone active'
,'Type of location'
,'Alignment'
,'Road surface'
,'Signals operation'
,'Speed limit'
]

date_time_varbs=[
'Month of crash'
,'Day of week of crash'
,'Two-hour intervals'
]

weather_varbs=[
'Surface condition',
'Weather',
'Natural lighting'
]

dependent_variable=['Degree of crash - detailed']


In [3]:
df_crashes=pd.read_csv('SydneyCrash.csv')


In [16]:
vul_users=pd.read_excel('NSW_Road_Crash_Data_2017-2021_TRAFFIC_UNIT.xlsx')
vul_users['Vul_user_presense']=np.where(vul_users['TU type group'].isin(['Motorcycle','Pedestrian','Pedal cycle','Non-motorised vehicle']),1,0)
vul_pvt=vul_users.pivot_table(index=['Crash ID'], values=['Vul_user_presense'],aggfunc='sum').reset_index()

In [47]:
new_dep_var_dict={1:"NoInjury",
                  2:"MinorInjury",
                  3:"MajorInjury_Fatal"}

In [26]:
severity_dict=dict(zip(df_crashes[dependent_variable].iloc[:,0].unique(),
         [5,2,1,3,4]))
df_crashes['DepVar']=df_crashes[dependent_variable[0]].map(severity_dict)
df_crashes['DepVar']=np.where(df_crashes['DepVar']>=4,3
                     ,np.where(df_crashes['DepVar']==1,1,2))
df_crashes['Vul_user_presense']=df_crashes['Crash ID'].map(dict(zip(vul_pvt['Crash ID'],vul_pvt['Vul_user_presense'])))

#df_x=df_crashes.loc[:,geom_varbs+date_time_varbs+weather_varbs+['DepVar']]
#dtale.show(df_x,
#          open_browser=True,ignore_duplicate=True)

In [27]:
from sklearn.preprocessing import Normalizer,MinMaxScaler
df_crashes['Speed limit_num']=pd.to_numeric(df_crashes['Speed limit'].str.strip(' km/h'), errors='coerce')
df_crashes['ModelVar_SpeedLimit']=MinMaxScaler()\
.fit_transform(np.array(df_crashes['Speed limit_num'].fillna(df_crashes['Speed limit_num'].mean()).astype(float)).reshape(-1, 1))
df_crashes['ModelVar_VulUser']=MinMaxScaler().fit_transform(np.array(df_crashes['Vul_user_presense']).reshape(-1, 1))
df_crashes['ModelVar_StreetType_HWY']=np.where(df_crashes['Street type']=='HWY',1,0)
df_crashes['ModelVar_StreetType_AVE']=np.where(df_crashes['Street type']=='AVE',1,0)
df_crashes['ModelVar_StreetType_AVE']=np.where(df_crashes['Street type']=='AVE',1,0)
df_crashes['ModelVar_FeatureType_HN']=np.where(df_crashes['Identifying feature type']=='HN',1,0)
df_crashes['ModelVar_LocationType_X_intersection']=np.where(df_crashes['Type of location']=='X-intersection',1,0)
df_crashes['ModelVar_HourInterval_LateNight']=np.where(df_crashes['Two-hour intervals'].isin(['22:00 - Midnight', '00:01 - 01:59']),1,0)
df_crashes['ModelVar_SurfaceCondition_Wet']=np.where(df_crashes['Surface condition']=='Wet',1,0)
df_crashes['ModelVar_Weather_Overcast']=np.where(df_crashes['Weather']=='Overcast',1,0)
df_crashes['ModelVar_Weather_Raining']=np.where(df_crashes['Weather']=='Raining',1,0)
df_crashes['ModelVar_NaturalLighting_Darkness']=np.where(df_crashes['Natural lighting']=='Darkness',1,0)
df_crashes['ModelVar_NaturalLighting_Dusk']=np.where(df_crashes['Natural lighting']=='Dusk',1,0)
df_crashes['ModelVar_SchoolZoneLocation_Yes']=np.where(df_crashes['School zone location']=='Yes',1,0)
df_crashes['ModelVar_SchoolZoneActive_Yes']=np.where(df_crashes['School zone active']=='Yes',1,0)
df_crashes['ModelVar_Alignment_Curved']=np.where(df_crashes['Alignment']=='Curved',1,0)
df_crashes['ModelVar_SignalsOperation_On']=np.where(df_crashes['Signals operation']=='On',1,0)
df_crashes['ModelVar_Month_May']=np.where(df_crashes['Month of crash']=='May',1,0)
df_crashes['ModelVar_Month_September']=np.where(df_crashes['Month of crash']=='September',1,0)
df_crashes['ModelVar_WeekDay_Weekend']=np.where(df_crashes['Day of week of crash'].isin(['Saturday','Sunday']),1,0)
df_crashes['ModelVar_SpeedLimit'].value_counts(normalize=False)

0.500000    20745
0.400000    20398
0.600000     7115
0.700000     4002
0.300000     2850
0.900000     1263
0.800000      507
1.000000      489
0.200000      116
0.000000       28
0.100000       21
0.495597        4
Name: ModelVar_SpeedLimit, dtype: int64

In [29]:
x_varbs=[f for f in df_crashes.columns if 'ModelVar_' in f]
y_varb=['DepVar']
x_train,x_test=(df_crashes[df_crashes['Reporting year']<=2020][x_varbs].reset_index(drop=True)
                ,df_crashes[df_crashes['Reporting year']>2020][x_varbs].reset_index(drop=True))

y_train,y_test=(df_crashes[df_crashes['Reporting year']<=2020][y_varb[0]].reset_index(drop=True)
                ,df_crashes[df_crashes['Reporting year']>2020][y_varb[0]].reset_index(drop=True))

### ORF

In [30]:
from orf import OrderedForest
# initiate the class with tuning parameters
oforest = OrderedForest(n_estimators=300, min_samples_leaf=20, honesty =True,
                        max_features=0.20,replace =False)
# fit the model
oforest.fit(X=x_train, y=y_train)
# predict ordered probabilities
oforest.predict(X=x_test)
# predict ordered classes
oforest.predict(X=x_test, prob=False)
# evaluate the prediction performance
oforest.performance()
# evaluate marginal effects
oforest.margins(X=x_train)

Prediction Performance of OrderedForest
--------------------------------------------------------------------------------
            mse       rps  accuracy
value  0.572761  0.165106  0.525641
--------------------------------------------------------------------------------


Confusion Matrix for OrderedForest
--------------------------------------------------------------------------------
                         Predictions 
--------------------------------------------------------------------------------
         Class 1  Class 2  Class 3
Class 1   3720.0  10894.0     89.0
Class 2   2268.0  20566.0    816.0
Class 3    824.0   7966.0   1042.0
--------------------------------------------------------------------------------


----------------------------------------------------------------------
Marginal Effects of OrderedForest, evaluation point: mean
----------------------------------------------------------------------
Effects:
---------------------------------------------------------

{'output': 'margin',
 'eval_point': 'mean',
 'X_eval': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18],
 'window': 0.1,
 'effects': array([[ 0.00061288, -0.02068321,  0.02007033],
        [-0.00018535,  0.00783376, -0.00764841],
        [ 0.01124925, -0.01762566,  0.00637641],
        [ 0.06162296, -0.05781084, -0.00381212],
        [-0.03172788,  0.02976901,  0.00195887],
        [ 0.04598057, -0.04428891, -0.00169166],
        [ 0.0271412 , -0.02082509, -0.00631611],
        [ 0.03658995, -0.04094225,  0.0043523 ],
        [ 0.01325428, -0.01671267,  0.00345839],
        [ 0.0411709 , -0.04916612,  0.00799521],
        [ 0.00960176,  0.01064186, -0.02024363],
        [ 0.00437286, -0.00691706,  0.00254421],
        [-0.00633408,  0.01117365, -0.00483957],
        [ 0.0010886 , -0.0434559 ,  0.0423673 ],
        [-0.04642854,  0.05358479, -0.00715626],
        [-0.00482699, -0.00296877,  0.00779576],
        [ 0.01752846,  0.00856324, -0.02609171],
        [ 0.00745

### SKlearn Models

In [32]:
param_grid = {'n_estimators': [15,35,70,141,210],
              'max_depth':np.arange(5,25,2),
              'max_features':('auto','log2'),
              'min_samples_split': (np.arange(0.01,0.21,0.02)*len(x_train)).astype(int)}
grid = GridSearchCV(ExtraTreesClassifier(),param_grid, cv=5, verbose=5,refit = True,n_jobs=-1) 
grid.fit(x_train, y_train)

Fitting 5 folds for each of 1000 candidates, totalling 5000 fits


GridSearchCV(cv=5, estimator=ExtraTreesClassifier(), n_jobs=-1,
             param_grid={'max_depth': array([ 5,  7,  9, 11, 13, 15, 17, 19, 21, 23]),
                         'max_features': ('auto', 'log2'),
                         'min_samples_split': array([ 481, 1445, 2409, 3372, 4336, 5300, 6264, 7227, 8191, 9155]),
                         'n_estimators': [15, 35, 70, 141, 210]},
             verbose=5)

In [40]:
grid.best_params_
ERT=ExtraTreesClassifier(max_depth= 15,
 max_features= 'log2',
 min_samples_split= 481,
 n_estimators= 141)
ERT.fit(x_train,y_train)
y_train_pred = ERT.predict(x_train)
y_test_pred = ERT.predict(x_test)


#ADAB=AdaBoostClassifier(n_estimators= 73,algorithm='SAMME')
#ADAB.fit(x_train,y_train)
#y_train_pred = ADAB.predict(x_train)
#y_test_pred = ADAB.predict(x_test)

print('accuracy train:  ' + str(sklearn.metrics.accuracy_score(y_train, y_train_pred)))
print('cf_matrix on train:    ' + str(sklearn.metrics.confusion_matrix(y_train, y_train_pred)))

print('accuracy test:   ' + str(sklearn.metrics.accuracy_score(y_test, y_test_pred)))
print('cf_matrix: on test:     ' + str(sklearn.metrics.confusion_matrix(y_test, y_test_pred)))

print('precision_score train:  ' + str(sklearn.metrics.precision_score(y_train, y_train_pred, average="weighted")))
print('precision_score test:   ' + str(sklearn.metrics.precision_score(y_test, y_test_pred, average="weighted")))

print('recall_score train:  ' + str(sklearn.metrics.recall_score(y_train, y_train_pred, average="weighted")))
print('recall_score test:   ' + str(sklearn.metrics.recall_score(y_test, y_test_pred, average="weighted")))

print('f1_score train:  ' + str(sklearn.metrics.f1_score(y_train, y_train_pred, average="weighted")))
print('f1_score test:   ' + str(sklearn.metrics.f1_score(y_test, y_test_pred, average="weighted")))

accuracy train:  0.5224447442150046
cf_matrix on train:    [[ 3752 10931    20]
 [ 2327 21218   105]
 [  902  8726   204]]
accuracy test:   0.5144873302683631
cf_matrix: on test:     [[ 804 2355    3]
 [ 495 3975   28]
 [ 165 1495   33]]
precision_score train:  0.5453001291302167
precision_score test:   0.5232960375594137
recall_score train:  0.5224447442150046
recall_score test:   0.5144873302683631
f1_score train:  0.4365834060679484
f1_score test:   0.4345692806329173


In [55]:
oforest_proba=pd.DataFrame(oforest.predict(X=x_test, prob=True)['predictions'], columns=[t+"_OForest" for t in new_dep_var_dict.values()])

In [56]:
y_test_pred_proba = ERT.predict_proba(x_test)
ert_probs=pd.DataFrame(y_test_pred_proba, columns=[t+"_ERT" for t in new_dep_var_dict.values()])

In [60]:
df_test=pd.concat([df_crashes[df_crashes['Reporting year']>2020].reset_index(drop=True),
                   ert_probs,oforest_proba],axis=1)

In [65]:
df_test['DependentVariable']=df_test.DepVar.map(new_dep_var_dict)

In [67]:
df_test.to_csv('model_results.csv')